<a href="https://colab.research.google.com/github/selvakumardhivakar/BookCom/blob/master/Colab-Dream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dream for Colab

Original repo: [dreamnettech/dreampower](https://github.com/dreamnettech/dreampower/tree/v1.2.10)

Colab: [FirstDee/dreamtime-v1.ipynb](https://gist.github.com/FirstDee/c3a94ff37e0561e597b65160aabbbc07)

This colab only points out how the output resolution can be configured and compressed the amount of cells. Thanks to FirstDee for creating the Colab.

# Project Setup
v1.2.10 contains --image-size, which can be applied in the last cell, but the output tends to look bad. Automatic mask creation seems to be pretty bad with this resolution. Only use this if you want to experiment. You may need to change overlay_size as well.

It may take a few minutes to load the program. Please be patient.

# Upload Images for Processing

# Upload Custom Mask (Advanced)

In [ ]:
#@title Upload Custom Mask

#@markdown You may upload a custom mask during runtime by checking the box below. This applies only if you upload one input image.
use_custom_maskfin = False #@param {type:"boolean"}

#@markdown The mask must be in PNG format in 512x512 size. [See this image for examples of each mask step](https://github.com/dreamnettech/dreampower/blob/v1.2.5/assets/transformation.jpg).
#@markdown
#@markdown When drawing your own mask, consider starting from an existing mask step, such as `DressToCorrect.png`. Run your input image normally, then look in the `/content/dreampower/work/altered` directory for existing masks.

from google.colab import files
import hashlib

#@markdown If you wish to use existing image steps, specify the `altered` directory here:
custom_altered_dir = '' #@param {type:'string'}
#@markdown Specify the step to begin processing. Steps with corresponding images:
#@markdown ```
#@markdown - :                    [ImageToCrop.png]
#@markdown - :                    [ImageToResized.png]
#@markdown 0 : dress -> correct   [DressToCorrect.png]
#@markdown 1 : correct -> mask    [CorrectToMask.png]
#@markdown 2 : mask -> maskref    [MaskToMaskref.png]
#@markdown 3 : maskref -> maskdet [MaskrefToMaskdet.png]
#@markdown 4 : maskdet -> maskfin [MaskdetToMaskfin.png]
#@markdown 5 : maskfin -> nude    [MaskfinToNude.png]
#@markdown - :                    [ImageToOverlay.png]
#@markdown ```
#@markdown These settings apply only if you are uploading a custom mask.
starting_step = 5 #@param {type:"integer"}
ending_step = 5 #@param {type:"integer"}
mask_n_runs = 1 #@param {type:"integer"}

input_mask_dir = ''

step_names = ['DressToCorrect.png','CorrectToMask.png','MaskToMaskref.png','MaskrefToMaskdet.png','MaskdetToMaskfin.png','MaskfinToNude.png']

if use_custom_maskfin:
  if starting_step < 1:
    raise('Must use custom mask step between 1 and 5. If using 0, then disable `use_custom_maskfin`.')
    
  if len(os.listdir(input_dir)) != 1:
    raise('Custom masks can only be used with one input image.')

  if os.path.isdir(custom_altered_dir):
    input_mask_dir = custom_altered_dir
  else:
    input_mask_dir = os.path.join(temp_dir, 'masks')
    if os.path.exists(input_mask_dir):
      shutil.rmtree(input_mask_dir)
    os.makedirs(input_mask_dir)

  # Construct altered subdir for input file
  # See processing/image.py line 34 v1.2.5
  input_fn = os.path.join(input_dir, os.listdir(input_dir)[0])
  input_basename = os.path.splitext(os.listdir(input_dir)[0])[0]
  input_hashedname = f'{input_basename}_{str(hashlib.md5(open(input_fn, "rb").read()).hexdigest())}'
  input_mask_subdir = os.path.join(input_mask_dir, input_hashedname)
  os.makedirs(input_mask_subdir, exist_ok=True)

  %cd "{temp_dir}"

  inputs = files.upload()

  for i, fn in enumerate(inputs.keys()):
    if not '.png' in fn.lower():
      raise('Mask must be in PNG format.')
    with open(os.path.join(input_mask_subdir, step_names[starting_step-1]), 'wb') as f:
      f.write(bytearray(inputs[fn]))
    
    if i < 5:
      display(Image(os.path.join(input_mask_subdir, step_names[starting_step-1]), width=512))

  if len(inputs) > 5:
      print('Displayed the first 5 images for processing.')

  # HACK: Copy Maskfin to the other step images, because dreampower expects them.
  # This has no effect on the MaskfinToNude step.
  if not os.path.isdir(custom_altered_dir):
    %cd "{input_mask_subdir}"
    shutil.copyfile(step_names[starting_step-1], 'ImageToCrop.png')
    shutil.copyfile(step_names[starting_step-1], 'ImageToResized.png')
    for i in range(0, starting_step-1):
      shutil.copyfile(step_names[starting_step-1], step_names[i])

# Processing